<a href="https://colab.research.google.com/github/larry-tableau/tableau/blob/main/Read_from_BQ_into_Hyper_via_Pantab_v3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BigQuery Data Extraction Tool

A Python-based tool for extracting large datasets from Google BigQuery with support for parallel processing and multiple output formats (Hyper, Parquet, CSV).

---

## Features

- Dynamic chunk sizing for optimal memory usage.
- Parallel processing using ThreadPoolExecutor.
- Support for Hyper, Parquet, and CSV file outputs.
- Detailed logging and error handling.
- Schema-based type conversion for BigQuery data.

---

## Requirements

- Python 3.8+
- Google Cloud SDK with authenticated credentials.

In [ ]:
!pip install google-auth google-auth-oauthlib google-auth-httplib2 google-cloud-bigquery pandas pantab==4.1.0



Will need to configure the relevant sections below
```
        # Configuration setup
        config = BigQueryConfig(
            project_id='pre-sales-demo',
            source_project='bigquery-public-data',
            dataset_id='google_trends',
            table_id='top_terms',
            output_format='hyper',
            output_path='./data',
            max_bytes_billed=1000 * 1024 * 1024 * 1024,  # 1TB
            initial_chunk_size=500_000,  # Start with smaller chunks
            max_workers=4,
            clean_up_temp_files=True
        )
```



In [ ]:
# Import required libraries
import subprocess
import sys
import os
import logging
import time
from datetime import datetime, timezone
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import pandas_gbq
import numpy as np
from typing import List, Optional, Tuple, Dict, Any
from dataclasses import dataclass
from IPython.display import clear_output, display, HTML
from concurrent.futures import ThreadPoolExecutor, as_completed
import psutil
import json
import pyarrow as pa
import pyarrow.parquet as pq
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed


# Install required packages
def install_requirements():
    """Install required packages silently."""
    requirements = [
        'pandas-gbq',
        'google-cloud-bigquery',
        'pyarrow',
        'pantab==4.1.0',
        'tableauHyperapi',
        'psutil'
    ]
    for package in requirements:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-q', package])

install_requirements()

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s'
)
logger = logging.getLogger(__name__)

class CoLabLogHandler(logging.Handler):
    """Custom log handler to store logs in memory for display."""
    def __init__(self, max_lines=1000):
        super().__init__()
        self.log_buffer = []
        self.max_lines = max_lines

    def emit(self, record):
        """Emit a log record."""
        log_entry = self.format(record)
        self.log_buffer.append(log_entry)
        if len(self.log_buffer) > self.max_lines:
            self.log_buffer = self.log_buffer[-self.max_lines:]

    def get_logs(self):
        """Get all stored logs."""
        return '\n'.join(self.log_buffer)

@dataclass
class BigQueryConfig:
    """Configuration for BigQuery extraction."""
    project_id: str
    source_project: str
    dataset_id: str
    table_id: str
    output_format: str = 'hyper'
    output_path: str = './data'
    max_bytes_billed: int = 100 * 1024 * 1024 * 1024  # 100GB
    initial_chunk_size: int = 500_000
    max_workers: int = 4
    columns: Optional[List[str]] = None
    where_clause: Optional[str] = None
    clean_up_temp_files: bool = True

    def __post_init__(self):
        """Validate configuration parameters."""
        if self.output_format not in ['hyper', 'parquet', 'csv']:
            raise ValueError("output_format must be one of: hyper, parquet, csv")
        if self.initial_chunk_size <= 0:
            raise ValueError("chunk_size must be positive")
        if self.max_workers <= 0:
            raise ValueError("max_workers must be positive")
        self.output_path = str(Path(self.output_path).resolve())

class BigQueryExtractor:
    """Enhanced BigQuery data extraction utility."""

    def __init__(self, config: BigQueryConfig):
        """Initialize extractor with configuration."""
        self.config = config
        self._ensure_output_directory()

        # Setup logging
        self.log_handler = CoLabLogHandler()
        self.log_handler.setFormatter(
            logging.Formatter('%(asctime)s - %(levelname)s - %(message)s',
                            datefmt='%Y-%m-%d %H:%M:%S')
        )
        logger.addHandler(self.log_handler)

        self.client = self._initialize_client()
        self.schema = self._get_schema()
        self.total_rows = 0
        self.processed_chunks = 0
        self.failed_chunks = []
        self.start_time = time.time()
        self.chunk_size = self.config.initial_chunk_size
        self._setup_progress_display()

    def _ensure_output_directory(self):
        """Create output directory if it doesn't exist."""
        os.makedirs(self.config.output_path, exist_ok=True)

    def _initialize_client(self) -> bigquery.Client:
        """Initialize BigQuery client with authentication."""
        try:
            auth.authenticate_user()
            return bigquery.Client(project=self.config.project_id)
        except Exception as e:
            logger.error(f"Failed to initialize BigQuery client: {str(e)}")
            raise

    def _get_schema(self) -> List[bigquery.SchemaField]:
        """Get table schema information."""
        try:
            dataset_ref = self.client.dataset(self.config.dataset_id,
                                            project=self.config.source_project)
            table_ref = dataset_ref.table(self.config.table_id)
            return self.client.get_table(table_ref).schema
        except Exception as e:
            logger.error(f"Failed to get schema: {str(e)}")
            raise

    def _get_bq_type_mapping(self) -> Dict[str, Any]:
        """Get mapping of BigQuery data types to Python/Pandas types."""
        return {
            'STRING': str,
            'BYTES': str,
            'INTEGER': 'Int64',
            'INT64': 'Int64',
            'FLOAT': 'float64',
            'FLOAT64': 'float64',
            'NUMERIC': 'float64',
            'BIGNUMERIC': 'float64',
            'BOOLEAN': 'boolean',
            'BOOL': 'boolean',
            'DATE': 'datetime64[ns]',
            'DATETIME': 'datetime64[ns]',
            'TIME': str,
            'TIMESTAMP': 'datetime64[ns]',
            'RECORD': str,
            'STRUCT': str,
            'ARRAY': str,
            'GEOGRAPHY': str
        }

    def _setup_progress_display(self):
        """Initialize progress display styling."""
        display(HTML("""
        <style>
            .bq-progress {
                font-family: monospace;
                padding: 10px;
                border: 1px solid #ccc;
                border-radius: 4px;
                margin: 10px 0;
                background-color: #f8f9fa;
            }
            .progress-bar {
                color: #fff;
                background-color: #28a745;
                height: 20px;
                border-radius: 3px;
                transition: width 0.3s ease;
                text-align: center;
                line-height: 20px;
            }
            .log-container {
                font-family: monospace;
                padding: 10px;
                border: 1px solid #ddd;
                border-radius: 4px;
                margin: 10px 0;
                background-color: #f8f9fa;
                max-height: 200px;
                overflow-y: auto;
                white-space: pre-wrap;
                font-size: 12px;
            }
            .log-entry {
                margin: 2px 0;
            }
            .log-error { color: #dc3545; }
            .log-warning { color: #ffc107; }
            .log-info { color: #17a2b8; }
        </style>
        """))

    @staticmethod
    def adjust_chunk_size(schema: List[bigquery.SchemaField], available_memory: int) -> int:
        """
        Dynamically adjust chunk size based on memory and schema details.
        Adds a cap to avoid overly large chunk sizes.
        """
        # Estimated memory usage per data type in bytes
        type_memory_footprint = {
            'STRING': 100,        # Average string size
            'BYTES': 50,          # Binary data
            'INTEGER': 8,         # 64-bit integers
            'INT64': 8,           # 64-bit integers
            'FLOAT': 8,           # 64-bit floats
            'FLOAT64': 8,         # 64-bit floats
            'NUMERIC': 16,        # Numeric with higher precision
            'BIGNUMERIC': 32,     # Big numeric
            'BOOLEAN': 1,         # Boolean
            'DATE': 4,            # Dates
            'DATETIME': 8,        # Datetime objects
            'TIMESTAMP': 8,       # Timestamp
            'RECORD': 200,        # Nested structure (average)
            'STRUCT': 200,        # Nested structure (average)
            'ARRAY': 150,         # Array (average per element)
            'GEOGRAPHY': 1000     # Geography data
        }

        # Calculate total memory usage per row based on schema
        total_memory_per_row = sum(
            type_memory_footprint.get(field.field_type, 50) for field in schema
        )

        # Account for overhead and concurrency
        row_memory_with_overhead = total_memory_per_row * 1.1  # Add 10% buffer

        # Calculate max rows based on available memory
        max_rows = available_memory // row_memory_with_overhead

        # Cap chunk size to avoid excessive memory usage
        capped_chunk_size = min(max_rows, 100_000)  # Cap at 100,000 rows

        # Log intermediate values for debugging
        logger.info(f"Available memory: {available_memory} bytes")
        logger.info(f"Estimated memory per row: {row_memory_with_overhead:.2f} bytes")
        logger.info(f"Calculated chunk size: {max_rows} rows")
        logger.info(f"Capped chunk size: {capped_chunk_size} rows")

        # Ensure a minimum chunk size to avoid inefficient queries
        return max(10_000, int(capped_chunk_size))

    def _count_records(self) -> int:
        """Count total records in table."""
        query = f"""
        SELECT COUNT(*) as total
        FROM `{self.config.source_project}.{self.config.dataset_id}.{self.config.table_id}`
        """
        if self.config.where_clause:
            query += f" WHERE {self.config.where_clause}"

        df = pandas_gbq.read_gbq(query, project_id=self.config.project_id)
        total = int(df['total'].iloc[0])
        logger.info(f"Total records to process: {total:,}")
        return total

    def _build_query(self, offset: int) -> str:
        """Build optimized BigQuery query."""
        columns = self.config.columns or [field.name for field in self.schema]

        query = f"""
        SELECT {', '.join(columns)}
        FROM `{self.config.source_project}.{self.config.dataset_id}.{self.config.table_id}`
        """

        if self.config.where_clause:
            query += f" WHERE {self.config.where_clause}"

        # Simple LIMIT/OFFSET without ORDER BY to avoid memory issues
        query += f"""
        LIMIT {self.chunk_size}
        OFFSET {offset}
        """

        return query

    def _process_columns(self, df: pd.DataFrame) -> pd.DataFrame:
        """Process and convert column data types."""
        if df.empty:
            return df

        column_types = {field.name: field.field_type for field in self.schema}
        type_mapping = self._get_bq_type_mapping()

        for column in df.columns:
            try:
                bq_type = column_types.get(column, 'STRING')

                # Handle different types
                if bq_type in ['RECORD', 'STRUCT']:
                    df[column] = df[column].apply(lambda x: json.dumps(x) if x is not None else None)

                elif bq_type == 'ARRAY':
                    df[column] = df[column].apply(
                        lambda x: json.dumps(list(x)) if isinstance(x, (list, np.ndarray)) else
                                (json.dumps([x]) if x is not None else None)
                    )

                elif bq_type in ['DATE', 'DATETIME', 'TIMESTAMP']:
                    df[column] = pd.to_datetime(df[column], errors='coerce')

                elif bq_type in ['NUMERIC', 'BIGNUMERIC', 'FLOAT', 'FLOAT64']:
                    df[column] = pd.to_numeric(df[column], errors='coerce')

                elif bq_type in ['INTEGER', 'INT64']:
                    df[column] = df[column].astype('Int64', errors='ignore')

                elif bq_type in ['BOOLEAN', 'BOOL']:
                    df[column] = df[column].astype('boolean', errors='ignore')

                elif bq_type == 'STRING' and df[column].nunique() / len(df) < 0.5:
                    df[column] = df[column].astype('category')

            except Exception as e:
                logger.warning(f"Error processing column {column}: {str(e)}")
                try:
                    df[column] = df[column].astype(str)
                except:
                    df[column] = df[column].apply(lambda x: str(x) if x is not None else None)

        return df

    def _save_chunk(self, df: pd.DataFrame, chunk_num: int) -> str:
        """Save data chunk with enhanced type handling."""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        base_filename = f"{self.config.table_id}_chunk_{chunk_num}_{timestamp}"

        try:
            filename = f"{base_filename}.parquet"
            full_path = f"{self.config.output_path}/{filename}"

            # Convert to Arrow Table and write with compression
            table = pa.Table.from_pandas(df)
            pq.write_table(table, full_path, compression='snappy')  # Snappy compression

            logger.debug(f"Saved chunk {chunk_num} to {full_path}")
            return filename

        except Exception as e:
            logger.error(f"Failed to save chunk: {str(e)}")
            raise


    def _update_progress(self, total_chunks: int):
        """Update progress display with logs."""
        clear_output(wait=True)

        elapsed_time = time.time() - self.start_time
        progress = (self.processed_chunks / total_chunks) * 100

        # Calculate statistics
        rows_per_second = self.total_rows / elapsed_time if elapsed_time > 0 else 0
        remaining_chunks = total_chunks - self.processed_chunks
        estimated_remaining = (remaining_chunks * elapsed_time) / max(1, self.processed_chunks)
        memory_info = psutil.Process(os.getpid()).memory_info()
        memory_usage_mb = memory_info.rss / 1024 / 1024

        # Create progress display
        progress_html = f"""
        <div class="bq-progress">
            <div style="font-weight: bold;">Processing: {self.config.source_project}.{self.config.dataset_id}.{self.config.table_id}</div>
            <div style="margin: 10px 0;">
                <div style="width: 100%; background-color: #eee; border-radius: 3px;">
                    <div class="progress-bar" style="width: {min(100, progress)}%;">
                        {progress:.1f}%
                    </div>
                </div>
            </div>
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 10px;">
                <div>Chunks: {self.processed_chunks}/{total_chunks}</div>
                <div>Rows: {self.total_rows:,}</div>
                <div>Failed Chunks: {len(self.failed_chunks)}</div>
                <div>Memory: {memory_usage_mb:.1f} MB</div>
                <div>Rate: {rows_per_second:.1f} rows/sec</div>
                <div>Time Left: {estimated_remaining:.1f}s</div>
            </div>
        </div>
        <div class="log-container">
        """

        # Add log entries with color coding
        logs = self.log_handler.get_logs()
        for log_line in logs.split('\n'):
            if 'ERROR' in log_line:
                log_class = 'log-error'
            elif 'WARNING' in log_line:
                log_class = 'log-warning'
            elif 'INFO' in log_line:
                log_class = 'log-info'
            else:
                log_class = ''
            progress_html += f'<div class="log-entry {log_class}">{log_line}</div>'

        progress_html += "</div>"
        display(HTML(progress_html))

    def _fetch_and_save_chunk(self, offset: int, chunk_num: int) -> Optional[str]:
        """Fetch and save a single chunk with retry logic."""
        max_retries = 3
        original_chunk_size = self.chunk_size

        for attempt in range(max_retries):
            try:
                if attempt > 0:
                    logger.info(f"Retry attempt {attempt + 1} for chunk {chunk_num}")
                    self.chunk_size = max(10000, original_chunk_size // (2 ** attempt))

                query = self._build_query(offset)
                df_chunk = pandas_gbq.read_gbq(
                    query,
                    project_id=self.config.project_id,
                    configuration={
                        'query': {
                            'useQueryCache': True,
                            'maximumBytesBilled': self.config.max_bytes_billed,
                            'priority': 'BATCH'
                        }
                    }
                )

                if not df_chunk.empty:
                    # Changed from _process_special_columns to _process_columns
                    df_chunk = self._process_columns(df_chunk)
                    saved_file = self._save_chunk(df_chunk, chunk_num)
                    self.total_rows += len(df_chunk)
                    self.processed_chunks += 1
                    return saved_file

            except Exception as e:
                logger.error(f"Error in attempt {attempt + 1} for chunk {chunk_num}: {str(e)}")
                if attempt == max_retries - 1:
                    self.failed_chunks.append(chunk_num)
                    return None
                time.sleep(2 ** attempt)  # Exponential backoff
            finally:
                self.chunk_size = original_chunk_size

        return None

    def parallel_process_chunks(chunk_count: int, process_chunk_fn, *args):
        """
        Process chunks in parallel using ProcessPoolExecutor.
        :param chunk_count: Total number of chunks to process.
        :param process_chunk_fn: Function to process each chunk.
        :param args: Additional arguments for process_chunk_fn.
        """
        results = []
        with ProcessPoolExecutor() as executor:
            futures = {
                executor.submit(process_chunk_fn, chunk_num, *args): chunk_num
                for chunk_num in range(chunk_count)
            }

            for future in as_completed(futures):
                chunk_num = futures[future]
                try:
                    result = future.result()
                    results.append(result)
                except Exception as e:
                    logger.error(f"Chunk {chunk_num} failed with error: {e}")

        return results




    def _merge_to_final_format(self, saved_files: List[str]) -> str:
        """Merge chunks into final output format."""
        final_filename = f"{self.config.table_id}_complete_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

        try:
            if self.config.output_format == 'hyper':
                final_path = f"{self.config.output_path}/{final_filename}.hyper"
                import pantab

                logger.info("Starting merge to Hyper format...")
                # Read and combine chunks with memory-efficient approach
                dfs = []
                total_size = 0
                max_batch_size = 1000000  # Maximum rows per batch

                for file in saved_files:
                    if file:
                        df = pd.read_parquet(f"{self.config.output_path}/{file}")
                        dfs.append(df)
                        total_size += len(df)

                        # Process batch if size threshold reached
                        if total_size >= max_batch_size:
                            combined_df = pd.concat(dfs, ignore_index=True)
                            pantab.frame_to_hyper(combined_df, final_path, table=self.config.table_id)
                            dfs = []  # Clear processed dataframes
                            total_size = 0

                # Process remaining dataframes
                if dfs:
                    combined_df = pd.concat(dfs, ignore_index=True)
                    pantab.frame_to_hyper(combined_df, final_path, table=self.config.table_id)

            elif self.config.output_format == 'csv':
                final_path = f"{self.config.output_path}/{final_filename}.csv"
                # Process chunks one at a time for CSV
                first_chunk = True
                for file in saved_files:
                    if file:
                        df = pd.read_parquet(f"{self.config.output_path}/{file}")
                        df.to_csv(final_path, mode='w' if first_chunk else 'a',
                                header=first_chunk, index=False)
                        first_chunk = False

            elif self.config.output_format == 'parquet':
                final_path = f"{self.config.output_path}/{final_filename}.parquet"
                # For parquet, we'll use a similar batching approach as hyper
                dfs = []
                total_size = 0
                max_batch_size = 1000000

                for file in saved_files:
                    if file:
                        df = pd.read_parquet(f"{self.config.output_path}/{file}")
                        dfs.append(df)
                        total_size += len(df)

                        if total_size >= max_batch_size:
                            combined_df = pd.concat(dfs, ignore_index=True)
                            combined_df.to_parquet(final_path)
                            dfs = []
                            total_size = 0

                if dfs:
                    combined_df = pd.concat(dfs, ignore_index=True)
                    combined_df.to_parquet(final_path)

            # Clean up temporary files if requested
            if self.config.clean_up_temp_files:
                for file in saved_files:
                    if file:
                        try:
                            os.remove(f"{self.config.output_path}/{file}")
                            logger.debug(f"Removed temporary file: {file}")
                        except Exception as e:
                            logger.warning(f"Failed to remove temporary file {file}: {str(e)}")

            return f"{final_filename}.{self.config.output_format}"

        except Exception as e:
            logger.error(f"Error merging files: {str(e)}")
            raise

    def extract_data(self) -> Tuple[int, str]:
        """Main extraction method with parallel processing."""
        try:
            # Initialize and validate
            memory_info = psutil.virtual_memory()
            available_memory = memory_info.available
            self.chunk_size = self.adjust_chunk_size(self.schema, available_memory)

            total_records = self._count_records()
            if total_records == 0:
                logger.warning("No records found to extract")
                return 0, None

            total_chunks = (total_records + self.chunk_size - 1) // self.chunk_size
            saved_files = []

            logger.info(f"Starting extraction with {self.config.max_workers} workers")
            logger.info(f"Initial chunk size: {self.chunk_size:,} rows")
            logger.info(f"Total chunks: {total_chunks}")

            # Extract data in parallel
            with ThreadPoolExecutor(max_workers=min(self.config.max_workers, os.cpu_count() * 2)) as executor:
                futures = []
                for chunk_num in range(total_chunks):
                    offset = chunk_num * self.chunk_size
                    future = executor.submit(self._fetch_and_save_chunk, offset, chunk_num)
                    futures.append(future)

                    offset = chunk_num * self.chunk_size
                    future = executor.submit(self._fetch_and_save_chunk, offset, chunk_num)
                    futures.append(future)

                for future in as_completed(futures):
                    try:
                        saved_file = future.result()
                        if saved_file:
                            saved_files.append(saved_file)
                        self._update_progress(total_chunks)
                    except Exception as e:
                        logger.error(f"Error processing chunk: {str(e)}")

            # Handle final merging
            if saved_files:
                try:
                    final_file = self._merge_to_final_format(saved_files)
                    logger.info(f"Successfully created final file: {final_file}")
                    return self.total_rows, final_file
                except Exception as e:
                    logger.error(f"Failed to create final file: {str(e)}")
                    raise
            else:
                logger.warning("No data was extracted successfully")
                return 0, None

        except Exception as e:
            logger.error(f"Error during extraction: {str(e)}")
            raise


def extract_bigquery_data(config: BigQueryConfig) -> Tuple[int, Optional[str]]:
    """Main function to extract data from BigQuery."""
    extractor = BigQueryExtractor(config)
    return extractor.extract_data()

# Example usage
if __name__ == "__main__":
    try:
        # Configure extraction
        config = BigQueryConfig(
            project_id='pre-sales-demo',
            source_project='bigquery-public-data',
            dataset_id='samples',
            table_id='github_timeline',
            output_format='hyper',
            output_path='./data',
            initial_chunk_size=500_000,
            max_workers=4,
            max_bytes_billed=1000 * 1024 * 1024 * 1024,  # 1TB
            clean_up_temp_files=True
        )

        logger.info("Starting BigQuery data extraction")
        logger.info(f"Source: {config.source_project}.{config.dataset_id}.{config.table_id}")
        logger.info(f"Output format: {config.output_format}")

        # Extract data
        total_rows, final_file = extract_bigquery_data(config)

        # Log results
        logger.info("Extraction completed successfully")
        logger.info(f"Total rows processed: {total_rows:,}")
        logger.info(f"Final output file: {final_file}")

    except Exception as e:
        logger.error(f"Extraction failed: {str(e)}", exc_info=True)
        raise
    finally:
        logger.info("Process completed")
